# Scikit-Learn and Scikit-Fingerprints Demo

## Introduction

This notebook explores how the actively maintained Scikit-Fingerprints module can be used to blend BioInformatics with machine learning in Scikit-Learn

### Questions

How do we split data sets?
How can we make a machine learning model most useful for future testing?
What data can we extract from our test results?

### Learning Objectives

* __Create and test a machine learning model__
    * Import and split a data set
    * Convert mols to fingerprints
    * Train the model using 80% of the data
    * Test the model's accuracy with the other 20% of the data
* __Create a pipeline to:__
    * Use Smiles as input
    * Predict molecules

### Purpose

This notebook is designed to show how scientists can create machine learning models to aid with BioInformatics research without needing advanced knowledge of machine learning

## Libraries

A list of libraries that will need to be installed and imported to complete the tasks in the notebook.

| Library | Contents | Source |
| :-----: | :------- | :----- |
| sklearn | library for creating and working with machine learning models| [Scikit-Learn documentation homepage](https://scikit-learn.org/stable/) |
| skfp | library for working with molecular fingerprints with Scikit-Learn | [scikit-fingerprints on GitHub](https://github.com/scikit-fingerprints/scikit-fingerprints/tree/master) |
| rdkit | libarary for cheminformatics and machine-learning modules | [rdkit documentation](https://www.rdkit.org/) |

## Installation

These libraries will need to be installed in your computing environment to perform the tasks in this notebook.

To install from the command line on your computer, use this command (with the `json` library as the example):

`pip install json`

To install from within a Jupyter notebook or CoLab notebook, you need to type the same command in a coding cell, preceded by an exclamation point.

`!pip install json`

These libraries will be imported as they are needed over the course of this notebook.


## Notebook Contents

The next section of the notebook includes all of the raw code for this example. **Experienced coder** should use this as you see fit, either in this notebook or in your preferred environment.

For **novice and intermediate coders**, the code is divided into sequential coding cells that each perform one step in the process. This notebook includes the following steps:

1. Imports
2. Load Data
3. Convert Smiles to Mols - Split Training/Testing Data Sets
4. Convert to Fingerprints
5. Train Model and Predict Accuracy
6. Test Predictions
7. Create a Smiles Pipeline
8. Test Pipeline Predictions
9. Derive useful datapoints

In [ ]:
# Full block of raw code for EXPERIENCED CODERS
# scikit-learn imports
from sklearn.ensemble import RandomForestClassifier # machine learning algorithm
from sklearn.metrics import roc_auc_score # prediction accuracy scorer
from sklearn.pipeline import make_pipeline # pipeline creater

# scikit-fingerprints imports
from skfp.datasets.moleculenet import load_hiv # built in test dataset
from skfp.fingerprints import ECFPFingerprint # fingerprints type
from skfp.model_selection import scaffold_train_test_split # train - test data split method
from skfp.preprocessing import MolFromSmilesTransformer # SMILES to mol converter
# other imports
from rdkit import Chem # RdKit Chem module for manually creating Mol objects for testing

smiles_list, y = load_hiv() # load the data from the scikit-fingerprints module
print("SMILES:")
print(smiles_list[:3]) # display the first three SMILES loaded in the dataset
print()
print("Labels:")
print(y[:1000]) # display the first 1000 y values in the dataset

mol_from_smiles = MolFromSmilesTransformer() # instantiate a SMILES to mols converter
dataset_size = 5000 # cap the dataset to 5000 values for the sake of minimizing loading time
mols = mol_from_smiles.transform(smiles_list[:dataset_size]) # transform the SMILES to mols
mols_train, mols_test, y_train, y_test = scaffold_train_test_split(
    mols, y[:dataset_size], test_size=0.2
) # create training and testing subsets
print("Molecules:")
print(mols_train[:3]) # print the first three converted mols in the training set

ecfp_fp = ECFPFingerprint() # instantate a fingerprint creater
X_train = ecfp_fp.transform(mols_train) # convert the X_training dataset from mols to fingerprints
X_test = ecfp_fp.transform(mols_test) # convert the X_testing dataset from mols to fingerprints
print("ECFP fingerprints:")
print(X_train.shape) # print the width and height of the X_training dataset
print(X_train[:3]) # print the first three elements of the X_training dataset
print(type(X_train)) # print the type of the dataset

clf = RandomForestClassifier(random_state=0) # instatiate the machine learning algorithm
clf.fit(X_train, y_train) # fit the training dataset to the algorithm (all the actual machine learning happens in this line of code)

clf.score(X_test, y_test) # score the dataset based on the accuracy of predictions of the testing dataset

def predict(index): # function to display the accuracy of a prediction based on an index in the testing dataset
    print("prediction: " + str(clf.predict(X_test)[index]) + ", actual: " + str(y_test[index]))

# some sample tests to illustrate accuracy
predict(5)
predict(400)
predict(852)
predict(853)
predict(849)

smiles_pipeline = make_pipeline(
    MolFromSmilesTransformer(),
    ecfp_fp,
    clf,
) # create a pipeline to take SMILES strings, convert them to mols, convert the mols to fingerprints, and predict their value using the machine learning model we created

# 1 Test
mol = "O=C(Nc1ccc(C2=NCCN2)cc1)Nc1cccc(C(=O)Nc2ccc(C3=NCCN3)cc2)c1" # From HIV Test Dataset
print("Smiles String: " + mol)
print(smiles_pipeline.predict([mol])[0])

# 0 Test
mol = "CS(=O)(=O)NC(=O)c1cc(Oc2ccc(C(F)(F)F)cc2Cl)ccc1[N+](=O)[O-]" # From TOX21 Test Dataset
print("Smiles String: " + mol)
print(smiles_pipeline.predict([mol])[0])

# Manual Entry
mol = "O=C(Nc1cccc(C(=O)Nc2ccc(C3=NCCN3)cc2)c1)c1ccc(C2=NCCN2)cc1" # Random
print("Smiles String: " + mol)
print(smiles_pipeline.predict([mol])[0])

def perf_measure(y_actual, y_hat): # function to tally true positives, false positives, true negatives, and false negatives from the testing data
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

TP, FP, TN, FN = perf_measure(clf.predict(X_test), y_test) # find the tp, fp, tn, and fn values using the function
print(f"True Positives: {TP}\nFalse Positives: {FP}\nTrue Negatives: {TN}\nFalse Negatives: {FN}\n")

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print(f"Sensitivity, hit rate, recall, or true positive rate: {TPR}\nSpecificity or true negative rate: {TNR}\nPrecision or positive predictive value: {PPV}\nNegative predictive value: {NPV}\nFall out or false positive rate: {FPR}\nFalse negative rate: {FNR}\nFalse discovery rate: {FDR}\nOverall accuracy: {ACC}")

### Imports


In [ ]:
# scikit-learn imports
from sklearn.ensemble import RandomForestClassifier # machine learning algorithm
from sklearn.metrics import roc_auc_score # prediction accuracy scorer
from sklearn.pipeline import make_pipeline # pipeline creater

# scikit-fingerprints imports
from skfp.datasets.moleculenet import load_hiv # built in test dataset
from skfp.fingerprints import ECFPFingerprint # fingerprints type
from skfp.model_selection import scaffold_train_test_split # train - test data split method
from skfp.preprocessing import MolFromSmilesTransformer # SMILES to mol converter
# other imports
from rdkit import Chem # RdKit Chem module for manually creating Mol objects for testing

### Load Data


In [ ]:
smiles_list, y = load_hiv() # load the data from the scikit-fingerprints module
print("SMILES:")
print(smiles_list[:3]) # display the first three SMILES loaded in the dataset
print()
print("Labels:")
print(y[:1000]) # display the first 1000 y values in the dataset

### Convert Smiles to Mols - Split Training/Testing Data Sets


In [ ]:
mol_from_smiles = MolFromSmilesTransformer() # instantiate a SMILES to mols converter
dataset_size = 5000 # cap the dataset to 5000 values for the sake of minimizing loading time
mols = mol_from_smiles.transform(smiles_list[:dataset_size]) # transform the SMILES to mols
mols_train, mols_test, y_train, y_test = scaffold_train_test_split(
    mols, y[:dataset_size], test_size=0.2
) # create training and testing subsets
print("Molecules:")
print(mols_train[:3]) # print the first three converted mols in the training set

### Convert to Fingerprints


In [ ]:
ecfp_fp = ECFPFingerprint() # instantate a fingerprint creater
X_train = ecfp_fp.transform(mols_train) # convert the X_training dataset from mols to fingerprints
X_test = ecfp_fp.transform(mols_test) # convert the X_testing dataset from mols to fingerprints
print("ECFP fingerprints:")
print(X_train.shape) # print the width and height of the X_training dataset
print(X_train[:3]) # print the first three elements of the X_training dataset
print(type(X_train)) # print the type of the dataset

### Train Model and Predict Accuracy


In [ ]:
clf = RandomForestClassifier(random_state=0) # instatiate the machine learning algorithm
clf.fit(X_train, y_train) # fit the training dataset to the algorithm (all the actual machine learning happens in this line of code)

clf.score(X_test, y_test) # score the dataset based on the accuracy of predictions of the testing dataset

### Test Predictions

In [ ]:
def predict(index): # function to display the accuracy of a prediction based on an index in the testing dataset
    print("prediction: " + str(clf.predict(X_test)[index]) + ", actual: " + str(y_test[index]))

# some sample tests to illustrate accuracy
predict(5)
predict(400)
predict(852)
predict(853)
predict(849)

### Create a Smiles Pipeline

In [ ]:
smiles_pipeline = make_pipeline(
    MolFromSmilesTransformer(),
    ecfp_fp,
    clf,
) # create a pipeline to take SMILES strings, convert them to mols, convert the mols to fingerprints, and predict their value using the machine learning model we created

### Test Pipeline Predictions

In [ ]:
# 1 Test
mol = "O=C(Nc1ccc(C2=NCCN2)cc1)Nc1cccc(C(=O)Nc2ccc(C3=NCCN3)cc2)c1" # From HIV Test Dataset
print("Smiles String: " + mol)
print(smiles_pipeline.predict([mol])[0])

# 0 Test
mol = "CS(=O)(=O)NC(=O)c1cc(Oc2ccc(C(F)(F)F)cc2Cl)ccc1[N+](=O)[O-]" # From TOX21 Test Dataset
print("Smiles String: " + mol)
print(smiles_pipeline.predict([mol])[0])

# Manual Entry
mol = "O=C(Nc1cccc(C(=O)Nc2ccc(C3=NCCN3)cc2)c1)c1ccc(C2=NCCN2)cc1" # Random
print("Smiles String: " + mol)
print(smiles_pipeline.predict([mol])[0])

### Derive useful datapoints

In [ ]:
def perf_measure(y_actual, y_hat): # function to tally true positives, false positives, true negatives, and false negatives from the testing data
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

In [ ]:
TP, FP, TN, FN = perf_measure(clf.predict(X_test), y_test) # find the tp, fp, tn, and fn values using the function
print(f"True Positives: {TP}\nFalse Positives: {FP}\nTrue Negatives: {TN}\nFalse Negatives: {FN}\n")

In [ ]:
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

In [ ]:
print(f"Sensitivity, hit rate, recall, or true positive rate: {TPR}\nSpecificity or true negative rate: {TNR}\nPrecision or positive predictive value: {PPV}\nNegative predictive value: {NPV}\nFall out or false positive rate: {FPR}\nFalse negative rate: {FNR}\nFalse discovery rate: {FDR}\nOverall accuracy: {ACC}")